# Student Name: VIGNESH GANESAN
## Student ID: 23079924

**Task 1**


Class Food
\begin{align*}
    & Precision_{Yes} = \frac{800}{800 + 200} = 0.8 \\
    & Precision_{No} = \frac{500}{500 + 200} = 0.714 \\
    & Recall_{Yes} = \frac{800}{800 + 200} = 0.8 \\
    & Recall_{No} = \frac{500}{500 + 200} = 0.714 \\
    & F1_{Yes} = 2 \times \frac{0.8 \times 0.8}{0.8 + 0.8} = 0.8\\
    & F1_{No} = 2 \times \frac{0.714 \times 0.714}{0.714 + 0.714} = 0.714  
\end{align*}

\\

Class Drink
\begin{align*}
    & Precision_{Yes} = \frac{70}{70 + 30} = 0.7 \\
    & Precision_{No} = \frac{100}{100 + 30} = 0.769 \\
    & Recall_{Yes} = \frac{70}{70 + 30} = 0.7 \\
    & Recall_{No} = \frac{100}{100 + 30} = 0.769 \\
    & F1_{Yes} = 2 \times \frac{0.7 \times 0.7}{0.7 + 0.7} = 0.7\\
    & F1_{No} = 2 \times \frac{0.769 \times 0.769}{0.769 + 0.769} = 0.769  
\end{align*}

\\

\begin{align*}
    & Microaveraged Precision = \frac{800+70}{800+200+70+30} = \frac{870}{1100} = 0.790 \\
    & Microaveraged Recall = \frac{800+70}{800+200+70+30} = \frac{870}{1100} = 0.790 \\
    & Microaveraged F1 = 2 \times \frac{0.790 \times 0.790}{0.790 + 0.790} = 0.790
\end{align*}

\\

\begin{align*}
    & Macroaveraged Precision = \frac{0.8+0.7}{2} = 0.746 \\
    & Macroaveraged Recall = \frac{0.8+0.7}{2} = 0.746 \\
    & Macroaveraged F1 = 2 \times \frac{0.746 \times 0.746}{0.746 + 0.746} = 0.746
\end{align*}

\\

Microaveraged Metrics:

The microaveraged F1 score is calculated by considering all instances together, regardless of their class. It is the weighted average of the F1 scores, where each instance contributes equally.
In this case, the microaveraged F1 score is higher than the individual F1 scores for both "Food" and "Drink" classes. This is because the "Food" class has more instances, and its higher F1 score contributes more to the microaveraged result.

\\
Macroaveraged Metrics:

The macroaveraged F1 score is calculated by taking the average of the F1 scores for each class. Each class contributes equally, regardless of the number of instances.
The macroaveraged F1 score provides a balanced view across classes. It is slightly lower than the microaveraged F1 score because it considers the lower F1 score of the "Drink" class, giving equal importance to both classes.

\\
Class Imbalance Impact:

The "Food" class, being larger, has a more significant impact on the microaveraged metrics. If the "Food" class performs well, it dominates the microaveraged results.
The macroaveraged metrics, on the other hand, are more influenced by the performance of the smaller "Drink" class. If the "Drink" class has a lower F1 score, it brings down the macroaveraged F1 score.

\\
In summary, the difference between microaveraged and macroaveraged F1 scores is influenced by the class imbalance. The microaveraged F1 score is higher because it is dominated by the larger class, while the macroaveraged F1 score provides a more balanced assessment across classes.

**Task 2**

In [2]:
import re
from collections import Counter

In [5]:
def normalizeText(text):
  return re.findall(r'\b\w+\b', text.lower())

def prepareMetadata(trainingSet):
  megaDocPLUS = ''
  megaDocMINUS = ''
  class_word_counts = {}
  class_counts = {}
  PLUS_bow = {}
  MINUS_bow = {}

  for doc, label in trainingSet:
    words = normalizeText(doc)
    if label == '+':
      megaDocPLUS += ' '.join(words)
      megaDocPLUS += ' '
    elif label =='-':
      megaDocMINUS += ' '.join(words)
      megaDocMINUS += ' '

    for word in words:
      class_word_counts.setdefault(label, {})
      class_word_counts[label][word] = class_word_counts[label].get(word, 0) + 1

      class_counts.setdefault(label, 0)
      class_counts[label] += 1

  megaDocPLUS_split = megaDocPLUS.split()
  megaDocMINUS_split = megaDocMINUS.split()
  PLUS_bow = [megaDocPLUS_split.count(w) for w in megaDocPLUS_split]
  MINUS_bow = [megaDocMINUS_split.count(w) for w in megaDocMINUS_split]
  PLUS_bow = dict(zip(megaDocPLUS_split, PLUS_bow))
  MINUS_bow = dict(zip(megaDocMINUS_split, MINUS_bow))

  V = dict(Counter(PLUS_bow) + Counter(MINUS_bow))

  print('megaDocPLUS: ', ' '.join(normalizeText(megaDocPLUS)))
  print('megaDocMINUS: ', ' '.join(normalizeText(megaDocMINUS)))
  print()

  print('class_counts: ', class_counts)
  probPLUS = class_counts['+'] / (class_counts['+'] + class_counts['-'])
  probMINUS = class_counts['-'] / (class_counts['+'] + class_counts['-'])
  print('probPLUS: ', probPLUS)
  print('probMINUS: ', probMINUS)
  print()

  print('PLUS_BOW: ', PLUS_bow)
  print('MINUS_BOW: ', MINUS_bow)
  print()

  print('V: ', V)
  print('|V|: ', len(V.keys()))
  print()

  return probPLUS, probMINUS, PLUS_bow, MINUS_bow, V, class_counts, class_word_counts


def naiveBayesClassifier(trainingSet, testSet):
    probPLUS, probMINUS, PLUS_bow, MINUS_bow, V, class_counts, class_word_counts = prepareMetadata(trainingSet)
    total_docs = len(V)

    for test_doc, _ in testSet:
      print('Test document: ', test_doc)
      docProbPLUS = 0
      docProbMINUS = 0
      words = normalizeText(test_doc)
      wordConditionalProbPLUSSum = 0.0
      wordConditionalProbMINUSSum = 0.0
      for word in words:
        wordConditionalProbPLUS = (PLUS_bow.get(word, 0) + 1) / (class_counts.get('+', 0) + len(V))
        wordConditionalProbPLUSSum += wordConditionalProbPLUS
        wordConditionalProbMINUS = (MINUS_bow.get(word, 0) + 1) / (class_counts.get('-', 0) + len(V))
        wordConditionalProbMINUSSum += wordConditionalProbMINUS
        print(f"Word: '{word}' \t wordConditionalProbPLUS: {wordConditionalProbPLUS} \t wordConditionalProbMINUS: {wordConditionalProbMINUS}")

        # word_count = class_word_counts.get('+', {}).get(word, 0) + 1
        # class_word_count = class_counts.get('+', 0) + total_docs
        # docProbPLUS += (word_count / class_word_count)

        # word_count = class_word_counts.get('-', {}).get(word, 0) + 1
        # class_word_count = class_counts.get('-', 0) + total_docs
        # docProbMINUS += (word_count / class_word_count)
      # print(f"docProbPLUS = {docProbPLUS} \t docProbMINUS = {docProbMINUS}")

      docProbPLUS = probPLUS * wordConditionalProbPLUSSum
      docProbMINUS = probMINUS * wordConditionalProbMINUSSum
      print(f"docProbPLUS = {docProbPLUS} \t docProbMINUS = {docProbMINUS}")

      inferred_class = "PLUS" if docProbPLUS > docProbMINUS else "MINUS" if docProbPLUS < docProbMINUS else "NEUTRAL"
      print('Inferred Class: ', inferred_class)
      print('-'*50)

trainingSet = [
    ('just plain boring', '-'),
    ('entirely predictable and lacks energy', '-'),
    ('no surprises and very few laughs', '-'),
    ('very powerful', '+'),
    ('the most fun film of the summer', '+')
]
testSet = [
    ('predictable with no fun', '?')
]

naiveBayesClassifier(trainingSet, testSet)

megaDocPLUS:  very powerful the most fun film of the summer
megaDocMINUS:  just plain boring entirely predictable and lacks energy no surprises and very few laughs

class_counts:  {'-': 14, '+': 9}
probPLUS:  0.391304347826087
probMINUS:  0.6086956521739131

PLUS_BOW:  {'very': 1, 'powerful': 1, 'the': 2, 'most': 1, 'fun': 1, 'film': 1, 'of': 1, 'summer': 1}
MINUS_BOW:  {'just': 1, 'plain': 1, 'boring': 1, 'entirely': 1, 'predictable': 1, 'and': 2, 'lacks': 1, 'energy': 1, 'no': 1, 'surprises': 1, 'very': 1, 'few': 1, 'laughs': 1}

V:  {'very': 2, 'powerful': 1, 'the': 2, 'most': 1, 'fun': 1, 'film': 1, 'of': 1, 'summer': 1, 'just': 1, 'plain': 1, 'boring': 1, 'entirely': 1, 'predictable': 1, 'and': 2, 'lacks': 1, 'energy': 1, 'no': 1, 'surprises': 1, 'few': 1, 'laughs': 1}
|V|:  20

Test document:  predictable with no fun
Word: 'predictable' 	 wordConditionalProbPLUS: 0.034482758620689655 	 wordConditionalProbMINUS: 0.058823529411764705
Word: 'with' 	 wordConditionalProbPLUS: 0.034482

Task 3

In [ ]:
pip install textblob

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('brown')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
from textblob import TextBlob

def sentimentAnalyzer(inp_string):
    blob = TextBlob(inp_string)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    sentiment = "Positive Sentiment" if polarity > 0.1 else "Negative Sentiment" if polarity < -0.1 else "Neutral Sentiment"
    confidence = abs(polarity)
    return sentiment, confidence, polarity, subjectivity

inp_string_list = [
    "NLP is cool",
    "NLP is cool and useful",
    "NLP is hard",
    "NLP is hard and useless",
    "NLP stands for Natural Language Processing"
]

for inp_string in inp_string_list:
    sentiment, confidence, polarity, subjectivity = sentimentAnalyzer(inp_string)
    print("String: ", inp_string)
    print(f"Sentiment(polarity={polarity}, subjectivity={subjectivity})")
    print(sentiment)
    print("Subjectivity:", subjectivity)
    print("")


String:  NLP is cool
Sentiment(polarity=0.35, subjectivity=0.65)
Positive Sentiment
Subjectivity: 0.65

String:  NLP is cool and useful
Sentiment(polarity=0.32499999999999996, subjectivity=0.325)
Positive Sentiment
Subjectivity: 0.325

String:  NLP is hard
Sentiment(polarity=-0.2916666666666667, subjectivity=0.5416666666666666)
Negative Sentiment
Subjectivity: 0.5416666666666666

String:  NLP is hard and useless
Sentiment(polarity=-0.39583333333333337, subjectivity=0.37083333333333335)
Negative Sentiment
Subjectivity: 0.37083333333333335

String:  NLP stands for Natural Language Processing
Sentiment(polarity=0.1, subjectivity=0.4)
Neutral Sentiment
Subjectivity: 0.4

